<a href="https://colab.research.google.com/github/zachary013/Lab3-DL/blob/main/Atelier3_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part1**

In [ ]:
# Check if GPU is available
import torch
print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU device name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

# Install required libraries
!pip install scrapy beautifulsoup4 pyarabic farasapy transformers

GPU available: True
GPU device name: Tesla T4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.0 MB/s eta 0:00:00


In [ ]:
# Web scraping with BeautifulSoup
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import random

# List of Arabic news websites
websites = [
    "https://www.aljazeera.net/news/politics",
    "https://www.bbc.com/arabic",
    "https://arabic.cnn.com/",
    # Add more sites as needed
]

collected_texts = []

# Scrape text from each website
for website in websites[:2]:  # Limiting to first 2 for demonstration
    try:
        response = requests.get(website, timeout=10)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract paragraphs (adjust selectors based on website structure)
        paragraphs = soup.find_all("p")

        for p in paragraphs:
            text = p.get_text().strip()
            # Filter for Arabic text with decent length
            if len(text) > 100 and re.search(r'[\u0600-\u06FF]', text):
                collected_texts.append(text)
    except Exception as e:
        print(f"Error scraping {website}: {e}")

# Create a dataset with assigned scores (simulated for this example)
data = []
for text in collected_texts[:20]:  # Limiting to first 20 texts
    # Assign a random score between 0 and 10 for demonstration
    score = round(random.uniform(0, 10), 1)
    data.append({"text": text, "score": score})

# Create DataFrame
df = pd.DataFrame(data)
print(f"Collected {len(df)} texts")
df.head()

# Save to CSV
df.to_csv('arabic_texts.csv', index=False)

Collected 20 texts


In [ ]:
# NLP Preprocessing Pipeline
import re
import numpy as np
from pyarabic import araby
from farasa.stemmer import FarasaStemmer

# Initialize Farasa stemmer
stemmer = FarasaStemmer()

def preprocess_arabic_text(text):
    # Remove diacritics
    text = araby.strip_tashkeel(text)

    # Remove non-Arabic characters and extra spaces
    text = re.sub(r'[^\u0600-\u06FF\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenization
    tokens = araby.tokenize(text)

    # Stemming
    stems = [stemmer.stem(token) for token in tokens]

    # Remove Arabic stop words (you might need a comprehensive list)
    arabic_stop_words = set(['من', 'إلى', 'في', 'على', 'عن', 'مع', 'هذا', 'هذه', 'ذلك', 'تلك'])
    filtered_stems = [token for token in stems if token not in arabic_stop_words]

    return " ".join(filtered_stems)

# Apply preprocessing to all texts
df['processed_text'] = df['text'].apply(preprocess_arabic_text)
df.head()

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [08:33<00:00, 470kiB/s]


In [ ]:
# Prepare dataset for deep learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# Tokenize and convert to indices
from collections import Counter

# Create vocabulary from processed texts
all_words = ' '.join(df['processed_text']).split()
word_counts = Counter(all_words)
vocabulary = {word: idx + 1 for idx, (word, _) in enumerate(word_counts.most_common())}
vocabulary['<PAD>'] = 0

# Convert texts to sequences of indices
def text_to_sequence(text, max_length=100):
    words = text.split()
    sequence = [vocabulary.get(word, 0) for word in words[:max_length]]
    if len(sequence) < max_length:
        sequence += [0] * (max_length - len(sequence))
    return sequence

df['sequence'] = df['processed_text'].apply(text_to_sequence)

# Normalize scores to [0, 1]
scaler = MinMaxScaler()
df['normalized_score'] = scaler.fit_transform(df[['score']])

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    list(df['sequence']),
    list(df['normalized_score']),
    test_size=0.2,
    random_state=42
)

# Create dataset and dataloader
class TextDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = [torch.tensor(seq, dtype=torch.long) for seq in sequences]
        self.labels = [torch.tensor([label], dtype=torch.float) for label in labels]

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

# Create DataLoaders
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Define model architectures

# 1. Simple RNN Model
class SimpleRNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SimpleRNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        return self.fc(hidden.squeeze(0))

# 2. Bidirectional RNN
class BiRNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(BiRNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        # Concatenate final forward and backward hidden states
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        return self.fc(hidden)

# 3. LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        return self.fc(hidden.squeeze(0))

# 4. GRU Model
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.gru(embedded)
        return self.fc(hidden.squeeze(0))

In [ ]:
# Training and evaluation function
def train_model(model, train_loader, test_loader, optimizer, criterion, epochs=10):
    model.to(device)

    train_losses = []
    test_losses = []

    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0

        for batch_idx, (texts, labels) in enumerate(train_loader):
            texts, labels = texts.to(device), labels.to(device)

            optimizer.zero_grad()
            predictions = model(texts)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # Evaluation phase
        model.eval()
        test_loss = 0
        predictions_list = []
        actual_list = []

        with torch.no_grad():
            for texts, labels in test_loader:
                texts, labels = texts.to(device), labels.to(device)
                predictions = model(texts)
                loss = criterion(predictions, labels)
                test_loss += loss.item()

                predictions_list.extend(predictions.cpu().numpy())
                actual_list.extend(labels.cpu().numpy())

        test_loss /= len(test_loader)
        test_losses.append(test_loss)

        print(f'Epoch: {epoch+1}')
        print(f'Train Loss: {train_loss:.4f}')
        print(f'Test Loss: {test_loss:.4f}')

        # Convert back to original scale for MSE calculation
        pred_orig = scaler.inverse_transform(np.array(predictions_list))
        actual_orig = scaler.inverse_transform(np.array(actual_list))
        mse = np.mean((pred_orig - actual_orig) ** 2)
        print(f'Test MSE (original scale): {mse:.4f}')
        print('-' * 60)

    return train_losses, test_losses

# Initialize model parameters
vocab_size = len(vocabulary)
embedding_dim = 100
hidden_dim = 128
output_dim = 1
learning_rate = 0.001
epochs = 10

# Train each model
models = {
    "Simple RNN": SimpleRNNModel(vocab_size, embedding_dim, hidden_dim, output_dim),
    "Bidirectional RNN": BiRNNModel(vocab_size, embedding_dim, hidden_dim, output_dim),
    "LSTM": LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim),
    "GRU": GRUModel(vocab_size, embedding_dim, hidden_dim, output_dim)
}

results = {}

for name, model in models.items():
    print(f"Training {name} model...")
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    train_losses, test_losses = train_model(
        model, train_loader, test_loader, optimizer, criterion, epochs
    )

    results[name] = {
        "train_losses": train_losses,
        "test_losses": test_losses,
        "model": model
    }

In [ ]:
# Model evaluation and metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

def evaluate_model(model, test_loader):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(device), labels.to(device)

            output = model(texts)

            predictions.extend(output.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

    # Convert back to original scale
    predictions = scaler.inverse_transform(np.array(predictions))
    actuals = scaler.inverse_transform(np.array(actuals))

    # Calculate metrics
    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)

    return {
        "MSE": mse,
        "MAE": mae,
        "R2": r2
    }

# Evaluate each model
evaluation_results = {}

for name, result in results.items():
    model = result["model"]
    metrics = evaluate_model(model, test_loader)
    evaluation_results[name] = metrics

    print(f"Evaluation for {name}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    print("-" * 60)

# Plot loss curves
plt.figure(figsize=(14, 8))

for i, (name, result) in enumerate(results.items()):
    plt.subplot(2, 2, i+1)
    plt.plot(result["train_losses"], label="Train Loss")
    plt.plot(result["test_losses"], label="Test Loss")
    plt.title(f"{name} Loss Curves")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

plt.tight_layout()
plt.show()

# Part2

In [ ]:
# Install necessary libraries
!pip install transformers datasets

# Import libraries
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
# Create a custom dataset for fine-tuning
import os

# Function to create a simple dataset
def create_custom_dataset(filename="custom_dataset.txt"):
    # Create a simple dataset with some text entries
    # You can replace this with your own dataset creation logic
    text_samples = [
        "مرحبا بك في عالم الذكاء الاصطناعي.",
        "تعلم الآلة هو مجال فرعي من الذكاء الاصطناعي.",
        "نماذج اللغة الكبيرة أحدثت ثورة في مجال معالجة اللغة الطبيعية.",
        "الشبكات العصبية العميقة قادرة على التعلم من البيانات.",
        "تستخدم خوارزميات التعلم العميق في العديد من التطبيقات.",
        # Add more sentences as needed
    ]

    # Write samples to a file
    with open(filename, 'w', encoding='utf-8') as f:
        for sample in text_samples:
            f.write(sample + "\n")

    return filename

# Create dataset file
dataset_file = create_custom_dataset()
print(f"Dataset created at: {dataset_file}")

In [ ]:
# Load pre-trained model and tokenizer
model_name = "gpt2"  # Can also use "gpt2-medium", "gpt2-large" or "gpt2-xl"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token to GPT2 tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Prepare dataset for fine-tuning
def load_dataset(train_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=block_size,
    )
    return dataset

# Load dataset
train_dataset = load_dataset(dataset_file, tokenizer)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Save the model
model_path = "./fine_tuned_gpt2"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"Model saved to {model_path}")

In [ ]:
# Generate text with the fine-tuned model
def generate_text(prompt, model, tokenizer, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generate text
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        do_sample=True,
        top_p=0.95,
        top_k=50,
        temperature=0.7,
        no_repeat_ngram_size=2,
    )

    # Decode and return the text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# Generate text
test_prompt = "مرحبا بك في"
generated_text = generate_text(test_prompt, fine_tuned_model, fine_tuned_tokenizer)
print(f"Prompt: {test_prompt}")
print(f"Generated text: {generated_text}")

In [ ]:
# Evaluate the text generation
def evaluate_generation(model, tokenizer, prompts):
    results = []

    for prompt in prompts:
        generated = generate_text(prompt, model, tokenizer)
        results.append({
            "prompt": prompt,
            "generated": generated
        })

    return results

# Test prompts
test_prompts = [
    "الذكاء الاصطناعي هو",
    "تعلم الآلة يساعدنا في",
    "اللغة العربية لها"
]

# Run evaluation
evaluation = evaluate_generation(fine_tuned_model, fine_tuned_tokenizer, test_prompts)

# Print results
for result in evaluation:
    print(f"Prompt: {result['prompt']}")
    print(f"Generated: {result['generated']}")
    print("-" * 60)

In [ ]:
# Save the trained model and results
import json

# Save evaluation results
with open("text_generation_results.json", "w", encoding="utf-8") as f:
    json.dump(evaluation, f, ensure_ascii=False, indent=2)

# Save a sample of the generated text
with open("sample_generated_texts.txt", "w", encoding="utf-8") as f:
    for result in evaluation:
        f.write(f"Prompt: {result['prompt']}\n")
        f.write(f"Generated: {result['generated']}\n")
        f.write("-" * 60 + "\n")

# Export conclusion
conclusion = """
Lab 3 Summary:
1. Created an Arabic text classifier using various recurrent neural architectures (RNN, BiRNN, LSTM, GRU)
2. Fine-tuned a GPT-2 model for Arabic text generation
3. Compared the performance of different sequence models on the classification task
4. Successfully generated coherent Arabic text continuations

The most efficient model for the classification task was [insert best model here] based on the evaluation metrics.
For text generation, the fine-tuned GPT-2 model produced [describe quality] results.

Key learnings:
- Sequence models are effective for both classification and generation tasks
- Bidirectional models often capture more context than unidirectional ones
- Pre-trained transformer models can be fine-tuned effectively even with small datasets
"""

with open("lab3_conclusion.txt", "w") as f:
    f.write(conclusion)

print("Lab 3 completed successfully!")